# 🤖 Telegram Video Bot для нарезки видео на шотсы

Этот notebook запускает Telegram бота, который:
- Скачивает видео с YouTube
- Нарезает их на короткие клипы (шотсы)
- Загружает на Google Drive
- Добавляет заголовки и подзаголовки

## 📋 Требования:
1. Токен Telegram бота
2. Google OAuth токен (base64)
3. Репозиторий с кодом на GitHub

## 🔧 Установка зависимостей

In [ ]:
# Установка системных зависимостей
!apt update -qq
!apt install -y ffmpeg

# Проверяем установку ffmpeg
!ffmpeg -version | head -1

In [ ]:
# Клонируем репозиторий с ботом
import os

# Замените на ваш GitHub репозиторий
GITHUB_REPO = "https://github.com/YOUR_USERNAME/YOUR_REPO.git"

# Клонируем репозиторий
if not os.path.exists('telegram-video-bot'):
    !git clone {GITHUB_REPO} telegram-video-bot
else:
    print("Репозиторий уже клонирован")

# Переходим в папку проекта
%cd telegram-video-bot

In [ ]:
# Установка Python зависимостей
!pip install -q python-telegram-bot==20.7
!pip install -q yt-dlp==2023.12.30
!pip install -q ffmpeg-python==0.2.0
!pip install -q openai-whisper==20231117
!pip install -q google-api-python-client==2.108.0
!pip install -q google-auth-httplib2==0.1.1
!pip install -q google-auth-oauthlib==1.1.0
!pip install -q python-dotenv==1.0.0
!pip install -q Pillow==10.1.0
!pip install -q moviepy==1.0.3

print("✅ Все зависимости установлены!")

## ⚙️ Настройка конфигурации

In [ ]:
# Настройка переменных окружения
import os
from google.colab import userdata

# Получаем токены из Colab Secrets или вводим вручную
try:
    # Попытка получить из Colab Secrets
    TELEGRAM_BOT_TOKEN = userdata.get('TELEGRAM_BOT_TOKEN')
    GOOGLE_OAUTH_TOKEN_BASE64 = userdata.get('GOOGLE_OAUTH_TOKEN_BASE64')
    print("✅ Токены получены из Colab Secrets")
except:
    # Ввод вручную
    print("❌ Токены не найдены в Colab Secrets")
    print("Введите токены вручную:")
    
    TELEGRAM_BOT_TOKEN = input("Telegram Bot Token: ")
    GOOGLE_OAUTH_TOKEN_BASE64 = input("Google OAuth Token (base64): ")

# Создаем .env файл
env_content = f"""TELEGRAM_BOT_TOKEN={TELEGRAM_BOT_TOKEN}
GOOGLE_OAUTH_TOKEN_BASE64={GOOGLE_OAUTH_TOKEN_BASE64}
"""

with open('.env', 'w') as f:
    f.write(env_content)

print("✅ Файл .env создан")

### 🔐 Как добавить токены в Colab Secrets:

1. Нажмите на иконку 🔑 в левой панели Colab
2. Добавьте следующие секреты:
   - `TELEGRAM_BOT_TOKEN` - токен вашего Telegram бота
   - `GOOGLE_OAUTH_TOKEN_BASE64` - токен Google OAuth в формате base64

Это более безопасный способ хранения токенов!

## 🚀 Запуск бота

In [ ]:
# Проверяем импорты
try:
    import telegram
    import yt_dlp
    import ffmpeg
    import whisper
    from google.oauth2.credentials import Credentials
    from googleapiclient.discovery import build
    import moviepy
    from PIL import Image
    print("✅ Все модули импортированы успешно")
except ImportError as e:
    print(f"❌ Ошибка импорта: {e}")

In [ ]:
# Запуск бота
import asyncio
import nest_asyncio

# Разрешаем вложенные event loops для Colab
nest_asyncio.apply()

print("🤖 Запускаем Telegram бота...")
print("Для остановки нажмите кнопку 'Interrupt execution' или Ctrl+C")
print("="*50)

# Запускаем бота
!python run_bot.py

## 📱 Как использовать бота:

1. **Запустите бота** - отправьте `/start`
2. **Настройте параметры** (опционально):
   - `/duration 30` - длительность клипов в секундах
   - `/title Мой заголовок` - заголовок для видео
   - `/subtitle Подзаголовок` - подзаголовок для видео
3. **Отправьте видео**:
   - Ссылку на YouTube видео
   - Или загрузите видео файл
4. **Получите результат** - ссылки на обработанные клипы в Google Drive

## 🔧 Возможные проблемы:

- **Ошибка токена**: Проверьте правильность Telegram Bot Token
- **Ошибка Google Drive**: Убедитесь, что OAuth токен действителен
- **Ошибка ffmpeg**: Перезапустите ячейку с установкой системных зависимостей
- **Timeout**: Для длинных видео может потребоваться больше времени

## 📊 Мониторинг и логи

In [ ]:
# Проверка логов (если есть)
import os
import glob

# Ищем лог файлы
log_files = glob.glob('*.log')
if log_files:
    print("📋 Найденные лог файлы:")
    for log_file in log_files:
        print(f"  - {log_file}")
        
    # Показываем последние строки из последнего лога
    latest_log = max(log_files, key=os.path.getctime)
    print(f"\n📄 Последние 20 строк из {latest_log}:")
    !tail -20 {latest_log}
else:
    print("📋 Лог файлы не найдены")

In [ ]:
# Проверка временных файлов
import os

print("📁 Содержимое рабочей директории:")
for item in os.listdir('.'):
    if os.path.isfile(item):
        size = os.path.getsize(item)
        print(f"  📄 {item} ({size} bytes)")
    else:
        print(f"  📁 {item}/")

# Показываем временные папки
temp_dirs = [d for d in os.listdir('.') if d.startswith('temp_')]
if temp_dirs:
    print(f"\n🗂️ Временные папки: {len(temp_dirs)}")
    for temp_dir in temp_dirs[:5]:  # Показываем только первые 5
        files_count = len(os.listdir(temp_dir))
        print(f"  📁 {temp_dir}/ ({files_count} файлов)")

## 🧹 Очистка временных файлов

In [ ]:
# Очистка временных файлов
import os
import shutil
import glob

print("🧹 Очистка временных файлов...")

# Удаляем временные папки
temp_dirs = glob.glob('temp_*')
for temp_dir in temp_dirs:
    if os.path.isdir(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"  🗑️ Удалена папка: {temp_dir}")

# Удаляем временные видео файлы
temp_videos = glob.glob('temp_video_*.mp4') + glob.glob('youtube_video_*.mp4')
for temp_video in temp_videos:
    if os.path.isfile(temp_video):
        os.remove(temp_video)
        print(f"  🗑️ Удален файл: {temp_video}")

print("✅ Очистка завершена!")